In [1]:
inputs = []
data_file = './input.txt'
with open(data_file, 'r') as file:
    for line in file:
        inputs.append(line.strip().split(' '))

for idx, i in enumerate(inputs):
    # '-5' is not numeric to python, so just look at the last letter
    if i[-1][-1].isnumeric(): 
        inputs[idx][-1] = int(i[-1])
        


ins = []
for i in inputs:
    if i[0]=='inp':
        ins.append([i,])
    else:
        ins[-1].append(i)
        
inputs = ins
inputs[0]

[['inp', 'w'],
 ['mul', 'x', 0],
 ['add', 'x', 'z'],
 ['mod', 'x', 26],
 ['div', 'z', 1],
 ['add', 'x', 14],
 ['eql', 'x', 'w'],
 ['eql', 'x', 0],
 ['mul', 'y', 0],
 ['add', 'y', 25],
 ['mul', 'y', 'x'],
 ['add', 'y', 1],
 ['mul', 'z', 'y'],
 ['mul', 'y', 0],
 ['add', 'y', 'w'],
 ['add', 'y', 12],
 ['mul', 'y', 'x'],
 ['add', 'z', 'y']]

In [2]:
class arithmeticLogicUnit:
    def __init__(self,inputs=None):
        self.amount = ord('w') # ascii value for 'w', useful for converting things
        
        self.inputPointer = 0
        self.outputs = [0,0,0,0]
        if inputs:
            self.setInputs(inputs)
        else:
            self.inputs = None
    
    def setPart(self,a,b):
        number1 = ord(a) - self.amount
        self.outputs[number1] = b
    
    def setInputs(self, inputs):
        ins = [int(i) for i in str(inputs)]
        if 0 in ins:
            raise ValueError("Cannot have '0' as an inputs")
        elif len(ins) != 14:
            raise ValueError(f"Must have 14 digits, got {len(ins)} - {ins}")
        self.inputPointer = 0  
        self.inputs = ins
        self.outputs = [0,0,0,0]
    
    # inp a - Read an input value and write it to variable a.
    def inp(self,a):
        number1 = ord(a) - self.amount
        
        self.outputs[number1] = self.inputs[self.inputPointer]
        self.inputPointer = self.inputPointer + 1
        
    # add a b - Add the value of a to the value of b, then store the result in variable a.
    def add(self,a,b):
        number1 = ord(a) - self.amount       
        if type(b)==int:
            self.outputs[number1] = self.outputs[number1] + b
        else:
            number2 = ord(b) - self.amount
            self.outputs[number1] = self.outputs[number1] + self.outputs[number2]
            
    # mul a b - Multiply the value of a by the value of b, then store the result in variable a.
    def mul(self,a,b):
        number1 = ord(a) - self.amount       
        if type(b)==int:
            self.outputs[number1] = self.outputs[number1] * b
        else:
            number2 = ord(b) - self.amount
            self.outputs[number1] = self.outputs[number1] * self.outputs[number2]        
             
    # div a b - Divide the value of a by the value of b, truncate the result to an integer, 
    # then store the result in variable a. (Here, "truncate" means to round the value toward zero.)
    def div(self,a,b):
        number1 = ord(a) - self.amount       
        if type(b)==int:
            if b == 0:
                raise ValueError(f"'b' must not equal 0, it is {b}")
            self.outputs[number1] = int(self.outputs[number1] / b)
        else:
            number2 = ord(b) - self.amount
            if self.outputs[number2] <= 0:
                raise ValueError(f"'b' must not equal 0, it is {self.outputs[number2]} for {b}")
                
            self.outputs[number1] = int(self.outputs[number1] / self.outputs[number2])
            
    # mod a b - Divide the value of a by the value of b, then store the remainder in 
    # variable a. (This is also called the modulo operation.)
    def mod(self,a,b):
        number1 = ord(a) - self.amount       
        if self.outputs[number1] < 0:
            raise ValueError(f"'a' must be greater or equal to 0, not {self.outputs[number1]} for {a}")
            
        if type(b)==int:
            if b <= 0:
                raise ValueError(f"'b' must be greater than 0, not {b}")            
            self.outputs[number1] = self.outputs[number1] % b
        else:
            number2 = ord(b) - self.amount
            if self.outputs[number2] <= 0:
                raise ValueError(f"'b' must be greater than 0, not {self.outputs[number2]} for {b}")
            self.outputs[number1] = self.outputs[number1] % self.outputs[number2]
            
    # eql a b - If the value of a and b are equal, then store the value 1 in variable 
    # a. Otherwise, store the value 0 in variable a.
    def eql(self,a,b):
        number1 = ord(a) - self.amount    
        if type(b)==int:
            self.outputs[number1] = (self.outputs[number1] == b) * 1
        else:
            number2 = ord(b) - self.amount
            self.outputs[number1] = (self.outputs[number1] == self.outputs[number2]) * 1
    
    def ret(self,a):
        number1 = ord(a) - self.amount
        return self.outputs[number1]

In [3]:
ALU = arithmeticLogicUnit()

In [281]:
len(inputs)

14

In [4]:
position = 0
outputs = [{
    'ins': 0,
    'outs': [0,0,0,0]
}]
for position, ins in enumerate(inputs[:2]):
    newOuts = []
    for idx, outs in enumerate(outputs):

        for num in range(1,10):
            # pre load previous values
            for b,a in enumerate(['w','x','y','z']):
                ALU.setPart(a,outs['outs'][b])

            #gets the position-ith bit starting from MSB
            number = (outs['ins'] // 10**(13-position)) - ((outs['ins'] // 10**(14-position))*10) + num
            if number > 9:
                continue

            for line in ins:
                if len(line) == 3:
                    cmd, a, b = line
                else:
                    cmd, a = line

                if cmd == 'inp':
                    ALU.setPart(a,number)
                    #ALU.inp(a)
                elif cmd == 'add':
                    ALU.add(a,b)
                elif cmd == 'mul':
                    ALU.mul(a,b)
                elif cmd == 'div':
                    ALU.div(a,b)
                elif cmd == 'mod':
                    ALU.mod(a,b)
                elif cmd == 'eql':
                    ALU.eql(a,b)

            canAppend = True
            for no in newOuts:
                if [ALU.ret('w'),ALU.ret('x'),ALU.ret('y'),ALU.ret('z')] == no['outs']:
                    canAppend = False
            
            if canAppend:
                newOuts.append({
                    'ins' : outs['ins']+ number*10**(13-position),
                    'outs' : [ALU.ret('w'),ALU.ret('x'),ALU.ret('y'),ALU.ret('z')]
                })

    outputs = newOuts
    print(len(outputs))
    
        


9
81


252

In [293]:
4433241415 % 26

5

In [5]:
newOuts

#w is number under consideration
#x is 1


[{'ins': 11000000000000, 'outs': [1, 1, 8, 346]},
 {'ins': 12000000000000, 'outs': [2, 1, 9, 347]},
 {'ins': 13000000000000, 'outs': [3, 1, 10, 348]},
 {'ins': 14000000000000, 'outs': [4, 1, 11, 349]},
 {'ins': 15000000000000, 'outs': [5, 1, 12, 350]},
 {'ins': 16000000000000, 'outs': [6, 1, 13, 351]},
 {'ins': 17000000000000, 'outs': [7, 1, 14, 352]},
 {'ins': 18000000000000, 'outs': [8, 1, 15, 353]},
 {'ins': 19000000000000, 'outs': [9, 1, 16, 354]},
 {'ins': 21000000000000, 'outs': [1, 1, 8, 372]},
 {'ins': 22000000000000, 'outs': [2, 1, 9, 373]},
 {'ins': 23000000000000, 'outs': [3, 1, 10, 374]},
 {'ins': 24000000000000, 'outs': [4, 1, 11, 375]},
 {'ins': 25000000000000, 'outs': [5, 1, 12, 376]},
 {'ins': 26000000000000, 'outs': [6, 1, 13, 377]},
 {'ins': 27000000000000, 'outs': [7, 1, 14, 378]},
 {'ins': 28000000000000, 'outs': [8, 1, 15, 379]},
 {'ins': 29000000000000, 'outs': [9, 1, 16, 380]},
 {'ins': 31000000000000, 'outs': [1, 1, 8, 398]},
 {'ins': 32000000000000, 'outs': [2,

In [ ]:
for number in range(99999970000000,11111111111111,-1):
    if number % 10000000 == 0:
        print(f"Doing {number}")
    
    try:    
        ALU.setInputs(number)

        for idx,line in enumerate(inputs):
            if len(line) == 3:
                cmd, a, b = line
            else:
                cmd, a = line

            if cmd == 'inp':
                ALU.inp(a)
            elif cmd == 'add':
                ALU.add(a,b)
            elif cmd == 'mul':
                ALU.mul(a,b)
            elif cmd == 'div':
                ALU.div(a,b)
            elif cmd == 'mod':
                ALU.mod(a,b)
            elif cmd == 'eql':
                ALU.eql(a,b)
            else:
                assert 3==2, f"Wrong command.  Got {line}"                 
    except:
        None
    else:
        if ALU.ret('z') == 0:
            print(f"Winning {number}")
            break

In [ ]:
number